<a href="https://colab.research.google.com/github/PraeJi/OCR_with_Qwen2VLModel/blob/main/OCR_image_qwen.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [22]:
from PIL import Image
import torch
from transformers import Qwen2VLForConditionalGeneration, AutoProcessor
import io
import os

In [3]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [4]:
print("model loading...")

%%capture
model = Qwen2VLForConditionalGeneration.from_pretrained(
    "Qwen/Qwen2-VL-2B-Instruct",
    torch_dtype = torch.float16,
    device_map = {"": device}
)
model.eval()
processor = AutoProcessor.from_pretrained("Qwen/Qwen2-VL-2B-Instruct")

print("model loaded")

`torch_dtype` is deprecated! Use `dtype` instead!


model loading...


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json: 0.00B [00:00, ?B/s]

model.safetensors.index.json: 0.00B [00:00, ?B/s]

...

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

chat_template.json: 0.00B [00:00, ?B/s]

model loaded


In [11]:
@torch.inference_mode()
def ocr_image(img):

  prompt = "Extract text from the image (OCR)"
  messages = [
      {
          "role": "user",
          "content": [
              {"type": "image"},
              {"type": "text", "text": prompt}
          ]
      }
  ]

  text_prompt = processor.apply_chat_template(messages, add_generation_prompt=True)
  inputs = processor(
      text = [text_prompt],
      images = [img],
      padding = True,
      return_tensors = "pt"
  )

  # move to GPU
  inputs = {k: v.to(device) for k,v in inputs.items()}

  # generate
  with torch.autocast("cuda"):
    output = model.generate(**inputs, max_new_tokens=512)

  generated = output[:, inputs["input_ids"].shape[1]:]

  text = processor.batch_decode(
      generated,
      skip_special_tokens = True
  )[0]

  return text

# For test ocr_image function

In [38]:
image_path = "/content/Screenshot 2025-11-29 212227.png"
result = ocr_image(image_path)

In [39]:
result

"Hello! I'm Jirapa (Prae)\n\nAI Engineer | Computer Science.\n\nPassionate about building and learning in the fields of Machine Learning and Deep Learning.\n\nBased in Thailand.\nBackground in Computer Science.\nMy Study Notes: MyStudyNotes\ncontact: jirapa.official@gmail.com\nAlways learning & exploring new technologies"

In [40]:
image_path1 = "https://media.istockphoto.com/id/914023158/vector/clean-minimal-invoice-vector-template-design.jpg?s=612x612&w=0&k=20&c=bUdGOlc4-xIrL6qyjN1P3izkfOM3sbb5EqYYiPJCPrc="
result_url = ocr_image(image_path1)

In [41]:
result_url

'Invoice\n\nTo: Dwayne Clark\n\n24 Dummy Street Area, Location Ipsum, 570x59x\n\nInvoice# 52168\n\nDate: 01/02/2020\n\nItem Description\n\n1. Lorem Ipsum\n2. Pellentesque id neque ligula\n3. interdum et malesuada fames\n4. Vivamus volutpat faucibus\n5. Vivamus volutpat faucibus\n6. Vivamus volutpat faucibus\n7. Vivamus volutpat faucibus\n8. Vivamus volutpat faucibus\n9. Vivamus volutpat faucibus\n10. Vivamus volutpat faucibus\n11. Vivamus volutpat faucibus\n12. Vivamus volutpat faucibus\n13. Vivamus volutpat faucibus\n14. Vivamus volutpat faucibus\n15. Vivamus volutpat faucibus\n16. Vivamus volutpat faucibus\n17. Vivamus volutpat faucibus\n18. Vivamus volutpat faucibus\n19. Vivamus volutpat faucibus\n20. Vivamus volutpat faucibus\n21. Vivamus volutpat faucibus\n22. Vivamus volutpat faucibus\n23. Vivamus volutpat faucibus\n24. Vivamus volutpat faucibus\n25. Vivamus volutpat faucibus\n26. Vivamus volutpat faucibus\n27. Vivamus volutpat faucibus\n28. Vivamus volutpat faucibus\n29. Vivamus